In [1]:
from astropy.io import fits
import pyregion
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u
from astropy import constants as cte 
import regions
from astropy.wcs import WCS
import cmasher as cm
import seaborn as sn
from discrete_gaussian_model import DiscreteGaussianModel
from astropy.nddata import Cutout2D
from astropy.coordinates import Angle
from astropy.modeling import models, fitting
from astropy.modeling.models import Const1D
from astropy.table import QTable, Table
from astropy.coordinates import SkyCoord
FITTER = fitting.LevMarLSQFitter()

In [40]:
G1 = np.array([
    1
])
G2 = np.array([
    1.5,-4
])
G3 = np.array([
    1.5,9,15,-2,-8
])
G4 = np.array([
    2,4,-2,-12
])
G5 = np.array([
    2,10,19,-2,-8,-13
])
G6 = np.array([
    2.5,-1,-9,-18
])

In [56]:
Rend = np.array([-18,-11.3,-5.1,0,9.4,16.5,23.8])

In [6]:
def dis(x,y):
    d = np.sqrt(x**2+y**2)
    return(d)

In [26]:
print(dis(G1,Rend[0]),np.arctan(-1*G1+Rend[0])%360)

[18.02775638] [358.48178673]


In [59]:
dis(G2,Rend[1])

array([11.39912277, 11.98707637])

In [60]:
dis(G3,Rend[2])

array([ 5.31601354, 10.34456379, 15.84329511,  5.47813837,  9.48736001])

In [61]:
dis(G4,Rend[3])

array([ 2.,  4.,  2., 12.])

In [62]:
dis(G5,Rend[4])

array([ 9.61041102, 13.72443077, 21.19811312,  9.61041102, 12.3434193 ,
       16.0424437 ])

In [63]:
dis(G6,Rend[5])

array([16.68831927, 16.53027525, 18.79494613, 24.41823089])

In [108]:
def PA(x,y):
    Pa = (atan2(y,x)*(180/np.pi))%360
    return(Pa)

In [107]:
from math import atan2
(atan2(Rend[0],G2[0])*(180/np.pi))%360

274.76364169072616

In [70]:
np.arctan(-1*G2+Rend[1])%360

array([358.50717031, 358.56534263])

In [71]:
print(dis(G1,Rend[0]),np.arctan(-1*G1+Rend[0])%360)

[18.02775638] [358.48178673]


In [72]:
from dataclasses import dataclass

@dataclass
class GlobuleGroup:
    """A spatial group of globules defined as a rectangle in PA, Sep space"""
    name: str
    pa0: u.Quantity["angle"]
    sep0: float
    dpa: u.Quantity["angle"] = 60 * u.deg
    dsep: float = 5.0

    def contains(self, pa, sep) -> bool:
        """Is a given PA, and Sep inside the box or not?"""
        Dsep = abs(sep - self.sep0)
        sep_check =  Dsep <= self.dsep / 2
        Dpa = abs(Angle(pa - self.pa0).wrap_at(180 * u.deg)) 
        pa_check = Dpa <= self.dpa / 2
        # print(Dsep, sep_check, Dpa, pa_check)
        return sep_check and pa_check


globule_groups = [
    GlobuleGroup("NE-04", 15 * u.deg, 3.5, 150 * u.deg,  7.0),
    GlobuleGroup("S-07", 200 * u.deg, 7.5, 100 * u.deg,  4.0),
    GlobuleGroup("W-08", 270 * u.deg, 9.0, 60 * u.deg,  6.0),
    GlobuleGroup("N-10", 350 * u.deg, 9.5, 60 * u.deg,  5.0),
    GlobuleGroup("S-12", 195 * u.deg, 12.0, 90 * u.deg,  6.0),
    GlobuleGroup("E-13", 70 * u.deg, 12.0, 100 * u.deg,  11.0),
    GlobuleGroup("N-16", 350 * u.deg, 17.0, 60 * u.deg,  8.0),
    GlobuleGroup("E-20", 90 * u.deg, 20.0, 40 * u.deg,  6.0),
    GlobuleGroup("S-20", 200 * u.deg, 20.0, 100 * u.deg,  8.0),
    GlobuleGroup("W-21", 280 * u.deg, 21.0, 40 * u.deg,  8.0),
    GlobuleGroup("E-27", 90 * u.deg, 29.0, 60 * u.deg,  8.0),
    GlobuleGroup("SW-27", 210 * u.deg, 27.0, 120 * u.deg,  6.0),
    GlobuleGroup("N-27", 340 * u.deg, 26.0, 60 * u.deg,  6.0),
    GlobuleGroup("S-33", 160 * u.deg, 34.0, 40 * u.deg,  4.0),

]

def assign_to_group(pa: u.Quantity["angle"], sep: u.Quantity["angle"]) -> str:
    for _group in globule_groups:
        if _group.contains(pa, sep.to(u.arcsec).value):
            # The first match is the one we use
            return _group.name
    # Case of no match
    return None

In [109]:
for i in range(len(G1)):
    print(assign_to_group(PA(G1[i],Rend[0])*u.deg,dis(G1[i],Rend[0])*u.arcsec))

W-21


In [110]:
for i in range(len(G2)):
    print(assign_to_group(PA(G2[i],Rend[1])*u.deg,dis(G2[i],Rend[1])*u.arcsec))

W-08
W-08


In [111]:
for i in range(len(G3)):
    print(assign_to_group(PA(G3[i],Rend[2])*u.deg,dis(G3[i],Rend[2])*u.arcsec))

None
N-10
N-16
None
S-07


In [112]:
for i in range(len(G4)):
    print(assign_to_group(PA(G4[i],Rend[3])*u.deg,dis(G4[i],Rend[3])*u.arcsec))

NE-04
NE-04
None
S-12


In [113]:
for i in range(len(G5)):
    print(assign_to_group(PA(G5[i],Rend[4])*u.deg,dis(G5[i],Rend[4])*u.arcsec))

E-13
E-13
None
E-13
None
None


In [114]:
for i in range(len(G6)):
    print(assign_to_group(PA(G6[i],Rend[5])*u.deg,dis(G6[i],Rend[5])*u.arcsec))

E-13
E-13
None
None


# Mapeo 3D

Ahora vamos a utilizar el articulo de Zavala para tener una distribución mejor en cuanto a los glóbulos

Para esto tenemos que localizar los glóbulos que hay en cada rendija, con esto podemos tener
 * Su distancia
 * su PA
 * velocidad

Creemos que hay un par de cáscaras en las que se encuentran ciertos glóbulos por lo que vamos a comprobar.

In [115]:
from astropy.io import fits
import pyregion
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u
from astropy import constants as cte 
import regions
from astropy.wcs import WCS
import cmasher as cm
import seaborn as sn
from discrete_gaussian_model import DiscreteGaussianModel
from astropy.nddata import Cutout2D
from astropy.coordinates import Angle
from astropy.modeling import models, fitting
from astropy.modeling.models import Const1D
from astropy.table import QTable, Table
from astropy.coordinates import SkyCoord
FITTER = fitting.LevMarLSQFitter()

In [346]:
def dis(x,y):
    d = np.sqrt(x**2+y**2)
    return(d)

from math import atan2

def PA(x,y):
    Pa = (atan2(y,-x)%360)#*(180/np.pi))%360
    return(Pa)

In [118]:
Rendijas = [
    -53.6,-41.1,-34.4,-26.8,-18.0,-11.3,5.1,0,9.4,16.5,23.8,32.1,41.1,49.9
]
len(Rendijas)

14

In [122]:
Y = [
    1.64,1.98,2.33,2.67,3.02,3.36,3.70,4.05,4.39,4.74,5.08
]
Pos = [
    50,40,30,20,10,0,-10,-20,-30,-40,-50
]
len(Pos)

11

In [193]:
def POS_Y(num):
    ele = 0
    for i in range(len(Pos)):
        if num < Y[i]:
            ele = i-1
            lon = np.abs(round((Y[i]-Y[i-1])/10,3))
            p1 = round(Y[i]-num,3)
            val1 = round(p1/lon,3)
            res = round(Y[i]+val1,3)
            print(round(Pos[i]+val1,3),lon,p1,val1,res)

In [310]:
POS_Y(4.07)

-20.588 0.034 0.32 9.412 13.802
-20.857 0.035 0.67 19.143 23.883
-20.294 0.034 1.01 29.706 34.786


In [162]:
4.05+(0.034*5.588)

4.239992

In [140]:
4.39+.035*2

4.46

In [187]:
3.04+.004375*4.57

3.05999375

In [300]:
BY = [
    -32,-25.5,-1.76,-1.47,1.47,4.41,22.5
]
Bvs = [
    4,25.1,-16,6,-4.57,-7,14
]
len(BY) == len(Bvs)

True

In [301]:
CY = [
    24.4,27.35,8.52,18.28
]
Cvs =[
    20,12,34,34
]
len(CY) == len(Cvs)

True

In [302]:
DY = [
    -19.71,-18,-3.52,0.29,10.28,14,19.14
]
Dvs = [
    30,46,-18,36,34,30,22
]
len(DY)==len(Dvs)

True

In [303]:
EY = [
    -37.71,-33.14,-28.23,-25.88,-23.82,-26.47,-22.64,-19.14,-9.07,-4.11,-2.35,.88,3.52,9.70,15.71,16,18.57
]
Evs = [
    4,-2,-14,-18,-16,36,42,-30,-12,-10,40,34,34,-34,-34,26,24
]
len(EY)==len(Evs)

True

In [305]:
FY = [
    -32.85,-31.71,-25.29,-25.88,-21.17,-13.14,-13.14,-8.23,-3.82,0,1.76,8.82,7.35,9.41,14,22.35
]
Fvs = [
    0,16,-26,30,34,-34,34,44,-18,-26,38,-34,50,30,32,38
]
len(FY)==len(Fvs)

True

In [306]:
GY = [
    -35.42,-19.42,-12.85,-10.57,3.23,8.52,18,21.76
]
Gvs = [
    0,32,36,36,42,42,44,30
]
len(GY)==len(Gvs)

True

In [309]:
HY = [
    18.28,15.14,10,3.23,-3.23,-8.23,-12.57,-14.85,-17.4,-23.52,-35.42,-37.71
]
Hvs =[
    22,30,36,44,46,38,40,22,34,32,-2,2
]
len(HY)==len(Hvs)

True

In [311]:
IY = [
    -34.28,-27.64,-18,-20.58,-18.28,-18.57,-15.17,-10.85,-8.23,-2.94,4.41,20,23.82,24.7,26.47
]
Ivs =[
    8,-18,-34,22,14,34,36,44,44,48,48,24,22,6,20
]
len(IY)==len(Ivs)

True

In [312]:
JY = [
    21.17,7.35,-1.76,-7.35,-17.14,-21.76,-23.52,-30.57,-38.28
]
Jvs = [
    44,50,46,52,36,22,-12,-8,2
]
len(JY)==len(Jvs)

True

In [313]:
KY = [
    -22.35,-14.85,-12,-1.76
]
Kvs = [
    26,24,26,32
]
len(KY)==len(Kvs)

True

In [314]:
LY = [
    -29.41,-27.05,-20.88,-19.14,-14.85,-7.05
]
Lvs = [
    18,-2,20,-34,-34,-40
]
len(LY)==len(Lvs)

True

In [315]:
MY = [
    -22.94,-13.14,-6.76
]
Mvs =[
    0,-24,14
]
len(MY)==len(Mvs)

True

In [332]:
Bdis = []
for i in BY:
    Bdis.append(dis(i,Rendijas[1]))
                
Cdis = []
for i in CY:
    Cdis.append(dis(i,Rendijas[2]))
                
Ddis = []
for i in DY:
    Ddis.append(dis(i,Rendijas[3]))
                
Edis = []
for i in EY:
    Edis.append(dis(i,Rendijas[4]))
                
Fdis = []
for i in FY:
    Fdis.append(dis(i,Rendijas[5]))
                
Gdis = []
for i in GY:
    Gdis.append(dis(i,Rendijas[6]))
                
Hdis = []
for i in HY:
    Hdis.append(dis(i,Rendijas[7]))
                
Mdis = []
for i in MY:
    Mdis.append(dis(i,Rendijas[12]))
                
Ldis = []
for i in LY:
    Ldis.append(dis(i,Rendijas[11]))
                
Idis = []
for i in IY:
    Idis.append(dis(i,Rendijas[8]))
                
Jdis = []
for i in JY:
    Jdis.append(dis(i,Rendijas[9]))
                
Kdis = []
for i in KY:
    Kdis.append(dis(i,Rendijas[10]))
                

In [358]:
import pandas as pd
B = pd.DataFrame(
    {
        'Y' : BY,
        'Dist' : Bdis,
        'v' : Bvs,
        'X' : Rendijas[1],
        'PA' : [PA(Rendijas[1],i) for i in BY]
    }
)
C = pd.DataFrame(
    {
        'Y' : CY,
        'Dist' : Cdis,
        'v' : Cvs,
        'X' : Rendijas[2],
        'PA' : [PA(Rendijas[2],i) for i in CY]
    }
)
D = pd.DataFrame(
    {
        'Y' : DY,
        'Dist' : Ddis,
        'v' : Dvs,
        'X' : Rendijas[3],
        'PA' : [PA(Rendijas[3],i) for i in DY]
    }
)
E = pd.DataFrame(
    {
        'Y' : EY,
        'Dist' : Edis,
        'v' : Evs,
        'X' : Rendijas[4],
        'PA' : [PA(Rendijas[4],i) for i in EY]
    }
)
F = pd.DataFrame(
    {
        'Y' : FY,
        'Dist' : Fdis,
        'v' : Fvs,
        'X' : Rendijas[5],
        'PA' : [PA(Rendijas[5],i) for i in FY]
    }
)
G = pd.DataFrame(
    {
        'Y' : GY,
        'Dist' : Gdis,
        'v' : Gvs,
        'X' : Rendijas[6],
        'PA' : [PA(Rendijas[6],i) for i in GY]
    }
)
H = pd.DataFrame(
    {
        'Y' : HY,
        'Dist' : Hdis,
        'v' : Hvs,
        'X' : Rendijas[7],
        'PA' : [PA(Rendijas[7],i) for i in HY]
    }
)
I = pd.DataFrame(
    {
        'Y' : IY,
        'Dist' : Idis,
        'v' : Ivs,
        'X' : Rendijas[8],
        'PA' : [PA(Rendijas[8],i) for i in IY]
    }
)
J = pd.DataFrame(
    {
        'Y' : JY,
        'Dist' : Jdis,
        'v' : Jvs,
        'X' : Rendijas[9],
        'PA' : [PA(Rendijas[9],i) for i in JY]
    }
)
K = pd.DataFrame(
    {
        'Y' : KY,
        'Dist' : Kdis,
        'v' : Kvs,
        'X' : Rendijas[10],
        'PA' : [PA(Rendijas[10],i) for i in KY]
    }
)
L = pd.DataFrame(
    {
        'Y' : LY,
        'Dist' : Ldis,
        'v' : Lvs,
        'X' : Rendijas[11],
        'PA' : [PA(Rendijas[11],i) for i in LY]
    }
)
M = pd.DataFrame(
    {
        'Y' : MY,
        'Dist' : Mdis,
        'v' : Mvs,
        'X' : Rendijas[12],
        'PA' : [PA(Rendijas[12],i) for i in MY]
    }
)


In [359]:
H

,Y,Dist,v,X,PA
0,18.28,18.28,22,0,0.0
1,15.14,15.14,30,0,0.0
2,10.00,10.00,36,0,0.0
3,3.23,3.23,44,0,0.0
4,-3.23,3.23,46,0,180.0
5,-8.23,8.23,38,0,180.0
6,-12.57,12.57,40,0,180.0
7,-14.85,14.85,22,0,180.0
8,-17.40,17.40,34,0,180.0
9,-23.52,23.52,32,0,180.0


In [352]:
def PA(x,y):
    Pa = (atan2(y,x)*(180/np.pi))%360-90
    return(Pa)

In [353]:
for i in H['Y']:
    print(PA(Rendijas[7],i))

0.0
0.0
0.0
0.0
180.0
180.0
180.0
180.0
180.0
180.0
180.0
180.0


In [354]:
for i in M['Y']:
    print(PA(Rendijas[12],i))

240.8318922352608
252.27050445646375
260.659795229958
